# ESM

In [1]:
%cd ~/repo/protein-transfer

/home/t-fli/repo/protein-transfer


In [2]:
%load_ext blackcellmagic

In [3]:
from scr.utils import pickle_load

In [4]:
df = pickle_load("data/proeng/gb1/two_vs_rest.pkl")

In [5]:
df.head()

,sequence,target,set,validation,mut_name,mut_numb
0,MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYD...,1.000000,train,NaN,parent,1
1,MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGIDGEWTYD...,1.445905,train,NaN,V39I,1
2,MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGLDGEWTYD...,1.690164,train,NaN,V39L,1
3,MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGMDGEWTYD...,1.170550,train,NaN,V39M,1
4,MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVAGEWTYD...,2.401243,train,NaN,D40A,1


In [6]:
df_train = df.loc[(df["set"] == "train") & (df["validation"] != True)]
df_val = df.loc[(df["set"] == "train") & (df["validation"] == True)]
df_test = df.loc[(df["set"] == "test")]

len(df_train), len(df_val), len(df_test), len(df)

(381, 43, 8309, 8733)

In [7]:
from scr.encoding.encoding_classes import ESMEncoder

In [9]:
no_flat_encoder = ESMEncoder(
    encoder_name="esm1b_t33_650M_UR50S",
    embed_layer=0,
).encode(mut_seqs=list(df_val.sequence), mut_names=list(df_val.mut_name))
one_emb = next(no_flat_encoder)
one_emb[0].shape

Loading esm1b_t33_650M_UR50S using 0 layer embedding


Using cache found in /home/t-fli/.cache/torch/hub/facebookresearch_esm_main


No embedding flattening


(43, 265, 1280)

In [10]:
one_emb

(array([[[ 0.27928993, -0.00919111,  0.05307703, ...,  0.0052783 ,
          -0.08536895,  0.2144556 ],
         [ 0.86820096,  0.01390768, -0.03625386, ..., -0.07747608,
          -0.06879894,  0.10098855],
         [ 0.9904541 ,  0.02066673,  0.04346693, ..., -0.04951385,
           0.04328922, -0.09477672],
         ...,
         [ 0.01445957,  0.15244195,  0.13133049, ...,  0.00364488,
           0.50209504,  0.2052859 ],
         [ 0.1433187 ,  0.08901688,  0.06859887, ..., -0.04608465,
           0.14192118,  0.12139929],
         [-0.07774346,  0.03318178,  0.14130217, ..., -0.08603121,
           0.30741137,  0.01056057]],
 
        [[ 0.27928993, -0.00919111,  0.05307703, ...,  0.0052783 ,
          -0.08536895,  0.2144556 ],
         [ 0.86820096,  0.01390768, -0.03625386, ..., -0.07747608,
          -0.06879894,  0.10098855],
         [ 0.9904541 ,  0.02066673,  0.04346693, ..., -0.04951385,
           0.04328922, -0.09477672],
         ...,
         [ 0.01445957,  0.1524419

In [11]:
mean_flat_encoder = ESMEncoder(
    encoder_name="esm1b_t33_650M_UR50S",
    embed_layer=0,
).encode(mut_seqs=list(df_val.sequence), mut_names=list(df_val.mut_name),flatten_emb="mean")
one_mean_emb = next(mean_flat_encoder)
one_mean_emb[0].shape

Loading esm1b_t33_650M_UR50S using 0 layer embedding


Using cache found in /home/t-fli/.cache/torch/hub/facebookresearch_esm_main


(43, 1280)

In [12]:
one_mean_emb

(array([[-0.17739098, -0.00067187,  0.01667726, ...,  0.00655907,
          0.03614971,  0.09654725],
        [-0.1747657 , -0.00065019,  0.01660449, ...,  0.00632377,
          0.03780067,  0.09638622],
        [-0.1749173 , -0.00061091,  0.01679106, ...,  0.00663596,
          0.03946506,  0.09655192],
        ...,
        [-0.177018  , -0.00064966,  0.0166867 , ...,  0.00618463,
          0.03673364,  0.09675171],
        [-0.1757744 ,  0.00030468,  0.01655487, ...,  0.00629132,
          0.03641494,  0.09631334],
        [-0.17786627, -0.00057332,  0.01687178, ...,  0.00618563,
          0.03663646,  0.09616786]], dtype=float32),
 ['D40N',
  'V39E:D40Y',
  'V39F:V54G',
  'V39K:D40I',
  'V39L:D40A',
  'V39L:D40I',
  'V39L:D40R',
  'V39L:D40S',
  'V39L:D40V',
  'V39M:V54L',
  'V39M:D40Q',
  'V39M:D40T',
  'V39Q:D40H',
  'V39Q:D40S',
  'V39R:D40R',
  'V39R:D40Y',
  'V39T:G41K',
  'V39T:D40K',
  'V39T:D40T',
  'D40A:V54N',
  'D40A:V54T',
  'D40A:V54W',
  'G41A:V54A',
  'G41A:V54L',
  '

In [10]:
mean_batch_encoder = ESMEncoder(
    encoder_name="esm1b_t33_650M_UR50S",
    embed_layer=0,
).encode(mut_seqs=list(df_val.sequence), mut_names=list(df_val.mut_name), batch_size=4, flatten_emb="mean")
one_mean_emb = next(mean_batch_encoder)

Loading esm1b_t33_650M_UR50S using 0 layer embedding


Using cache found in /home/t-fli/.cache/torch/hub/facebookresearch_esm_main
  0%|          | 0/11 [00:00<?, ?it/s]

In [11]:
one_mean_emb[0].shape, len(one_mean_emb[1])

((4, 1280), 4)

In [14]:
one_mean_emb[1], df_val.mut_name[:4]

(['D40N', 'V39E:D40Y', 'V39F:V54G', 'V39K:D40I'],
 20          D40N
 56     V39E:D40Y
 60     V39F:V54G
 117    V39K:D40I
 Name: mut_name, dtype: object)

In [3]:
import torch

# Load ESM-1b model
original_model, alphabet = torch.hub.load(
            "facebookresearch/esm:main", 
            # model="esm1_t6_43M_UR50S"
            model="esm1b_t33_650M_UR50S"
        )
batch_converter = alphabet.get_batch_converter()
data = [
    ("protein1", "MSGEEEKAADFYVRYYVGHKGKFGHEFLEFEFRPNGSLRYANNSNYKNDTMIRKEATVSESVLSELKRIIEDSEIMQEDDDNWPEPDKIGRQELEILYKNEHISFTTGKIGALADVNNSKDPDGLRSFYYLVQDLKCLVFSLIGLHFKIKPI"),
    ("2", "VSDGMTGSNPVSPASSSSPDSSGAGGISPQHIAQDSSLDGPPGPQDGTTVPLEGFSLSHAADLVNRGQKWEKSHAEIAEQAKHEAEIETRIAELRKEGFWSLKRLPKVPEPPRPKGHWDYLCEEMQWLSADFAQERRWKRGVARKVVRMVIRHHEEQRQKEERARREEQAKLRRIASTMAKDVRQFWSNVEKVVQFKQQSRLEEKRKKALDLHLDFIVGQTEKYSDLLSQSLNQPPASSKAGSSPCLGSSSAASSPPPPVSRLDDEDGDFQPQEEEEEDDEETIEVEEQQEGNDAETQRREIELLRHEGELPLEELLRSLPPQLLGGPFSPSQTPSHDSDTQDGPEENIEEEPSQDLEVHPPSSAVTQCNKQRWHPDEDDEEFTANEDEAEDEEDTIAAEEQLEGEVDHAMELSELAREGELSMEELLQQYAGAYACDASAPASGDSEDEDEVEANSSDGELEETVEEAAQEDSSSQSDSAEECSEDEEDEHSEEEMSGSSQSEESESDESEDAQSQSQADEEQDDEGEDDDVDDDDDDGFGVEYLLARDDERSEVDGGSGPPTPGPTTTLGPKKEITDIAAAAESLQPKGYTLATTQVKTPIPLLLRGQLREYQHIGLDWLVTMYEKKLNGILADEMGLGKTIQTISLLAHLACEKGNWGPHLIIVPTSVMLNWEMELKRWCPSFKILTYYGAQKERKLKRQGWTKPNAFHVCITSYKLVLQDHQAFRRKNWRYLILDEAQNIKNFKSQRWQSLLNFNSQRRLLLTGTPLQNSLMELWSLMHFLMPHVFQSHREFKEWFSNPLTGMIEGSQEYNEGLVKRLHKVLRPFLLRRVKVDVEKQMPKKYEHVIRCRLSKRQRCLYDDFMAQTTTKETLATGHFMSVINILMQLRKVCNHPNLFDPRPVTSPFITPGICFSTASLVLRATEVHPLQRIDMGRFDLIGLEGRVSRYEADTFLPRHRLSHRILLEIATAPDPPPRPKPVKMKVNRMLQPVPKQEGRTVVVVNSPRPPLGPVPVRPPPGPEVSAQPALGPVPPVLPAPLMVSASPAGTPVVPASRPPGPVLLSPLQPNTGPLPQAGEVVSIGQLASLAQRPVTSAGGSKPLTFQIQGNKLTLTGAQVRQLAVGQPRPLQMPPSMVNNTGVVKIVVRQAPRDGLTPVPPLAPAPRPSSSGLPAVLTSRPTLTPSRLSSPTLGTTRTPIPTPTLVRPLLKLVHSPSPEVSASAPGAAPMTISSSLHMPSSLPGPASSPMTIHNSSPLSSPVSSTVSVPVSSSLPISVPTTLPVPTSAPLPIPITAPLPVSDLGPALLTSVTPALAVVSAASGPPLASAGVSPSMSALALGLATTPALSPPQTSGHSLLLATTSSHVPGLNSAVAPACSPVLVSASALTSPFPVSPSSAPAQASLLAAAPSTSQALATSLAPMAASQTAILGPSPTPPLAPLPVLAASQTPLPAMTPPSMSGTPLPSSSLVSAPTSVLAPSSTQTLVSTPVSSPLSSLASTQTLTLAPTLAPTLGGLSPSQTHSLGTGSPQGPFPIQTLSLTPASSLVPTPAQTLSLAPGGPVGPTQTLSLAPVPPLPPSSPVGPAPGHTLTLAPAASSASLLVPTSVQTLTLSPAPVPVPTLDLSATQTLALAPALTQAPASQASSLVSASGAASLPVTMVNRLPVPKDEPETLTLRSGPPSPLPTATSFSGPRPRRQPPPPPRSPFYLDSLEEKRKQQRSERLDRIFQLSEAHGALAPVYGTEVLDFCTLPQPVASPIGPRSPGPSHPIFWTYTEAARRAVLFPQQRLEQLSEIIERFIFVMPPVEAPPPSLHACHPPPWLAPHQAAFQEQLACELWPRARPLHRIVCNMRTQFPDLRLIQYDCGKLQTLAVLLRQLKAEGHRVLIFTQMTRMLDVLEQFLTYHGHLYLRLDGSTRVEQRQALMERFNADKRIFCFILSTRSGGVGVNLTGADTVVFYDSDWNPTMDAQAQDRCHRIGQTRDVHIYRLISERTVEENILKKANQKRMLGDMAIEGGNFTTAYFKQQTIRELFDMPLEEPPGSSVSSVPEDEEEAVASKQTHILEQALCRAEDEEDIRAATQAKAEQVAELAEFNENDGFPAGEGEEANRPGPGAEDEEMSRAEQEIAALVEQLTPIERYAMKFLEASLEEVSREELKQAEEQVEAARKDLDQAKEEVFRLPHEEEEGPGAGDEMSCGSSGASHRRSKKIRAPERLGTRVSERLRGARAETQGANHTPVTSSHHTCNTFVPPQCSPVRERIPRPVPRPRPPPPPPAPPPPPAPPASPSPAPPPPPAPASPAPPAPPASPPAASPSPAPLPPAASVASAITSALATAPIPISAPNPVTILPVHILPTPPLLQLPPSSSPASTPPSACTPLPTCTPLQAQTSVLTSSSPAPLDSLSVPISPGVTNLPLDSGEVGLCAQELAPVESLELPGMTNSEASLASLVSPKDLLPVAVDILPESEKTLPCIPAPSSTLEAESIPNGQQQEVPGPVEGTNTTLLPGSEELTMSLNESNGLELPPSAASAEPLDELLETHRNSGETREVQIPISNSEKPQELVRAEVTAPSTSSSATSSPEGPLPARPPRRRTSADVEIRGQGSGRTGQLPGPKVLRKLPGRLVTVVEEKELVKRRRHRAPASTLVPGVSEVSASPRSPSIHSMSGPESSPPSSGPCEAAPTSSLSTPTQSTFIACRHNELGTTTGGSPENGEGTELAITPPAVKRRRGRPPKKNRSPADAGRGVDEVPSSISKGKTNGVDPVPGPETVTVAEPDLRPQFVPGPQPPGPQPVHRPEPTILSPVEKRRRGRPPKARDLPIPGTISSPGGGNLESRTQSLPFPAPLTPLLACPTAVTNTVTTVTISTSPPKRKRGRPPKNPPSPRPSQHPVLDPDSSSILESCGLGVQRQPLGQAESEGSSSDEDGSRPLTRLARLRLEAEGMRGRKSEGSMVMAVIQDDLDLADSGPSGLELTPPVVSLTPKFRSTRLRPGSLVPPLETEKMPRKRSGAPVGSSGMAKRGRLQPPSPLGPEGSVEESEVEASGDEEEGDGTPRRRTGPRRLVGTTNQGDQRILRSSAPPHLCIPTISHRGRKAKT")
]
batch_labels, batch_strs, batch_tokens = batch_converter(data)

original_model.eval()
with torch.no_grad():
    original_results = original_model(batch_tokens, repr_layers=[6], return_contacts=False)
original_reps = original_results["representations"][6].numpy().mean(axis=1).flatten()

Using cache found in /home/t-fli/.cache/torch/hub/facebookresearch_esm_main
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm1b_t33_650M_UR50S.pt" to /home/t-fli/.cache/torch/hub/checkpoints/esm1b_t33_650M_UR50S.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm1b_t33_650M_UR50S-contact-regression.pt" to /home/t-fli/.cache/torch/hub/checkpoints/esm1b_t33_650M_UR50S-contact-regression.pt


ValueError: Sequence length 3097 above maximum  sequence length of 1024

In [12]:
batch_tokens.shape

torch.Size([2, 3096])